# Unified AI vs Real Image Dataset Builder

This notebook builds a **clean, labeled, unified dataset** for evaluating
AI image detection systems.

### Supported sources
- HuggingFace datasets (DiffusionDB, COCO, OpenImages)
- Kaggle public datasets (Midjourney, AI vs Real)
- Unified output format:
  - Normalized PNG images
  - Size-limited (≤1024px)
  - Central metadata CSV

### Output Structure

```bash
tests/dataset/
├── ai/
├── real/
├── raw_downloads/
├── metadata/dataset_index.csv
```

> ⚠️ All datasets used are **public & legally accessible**.


## Required Dependencies

Before running, ensure:

```bash
pip install datasets pillow tqdm kaggle pycocotools
```

Also configure Kaggle:

```bash
~/.kaggle/kaggle.json
```


## Imports & Config

In [1]:
# ===============================
# Imports & Global Configuration
# ===============================
import os
import csv
import uuid
import subprocess
from PIL import Image
from tqdm import tqdm
from pathlib import Path
from datasets import load_dataset


# ===============================
# Directory Configuration
# ===============================
BASE_DIR       = Path("../tests/dataset")
AI_DIR         = BASE_DIR / "ai"
REAL_DIR       = BASE_DIR / "real"
RAW_DIR        = BASE_DIR / "raw_downloads"
META_DIR       = BASE_DIR / "metadata"

META_FILE      = META_DIR / "dataset_index.csv"

TARGET_PER_DS  = 1000
IMAGE_SIZE_MAX = 1024


## Utility Functions

These helpers:
- Ensure directory structure
- Normalize images (RGB, resize, PNG)
- Write metadata rows safely

In [2]:
def ensure_dirs():
    for d in [AI_DIR, REAL_DIR, RAW_DIR, META_DIR]:
        d.mkdir(parents=True, exist_ok=True)


def normalize_and_save(image: Image.Image, path: Path):
    """
    Normalize image to RGB PNG and limit size
    """
    image = image.convert("RGB")
    image.thumbnail((IMAGE_SIZE_MAX, IMAGE_SIZE_MAX))
    image.save(path, 
               format   = "PNG", 
               optimize = True,
              )


def write_meta(writer, **row):
    writer.writerow(row)
    

## Dataset Registry

Defines **where data comes from** and **how it is labeled**.

In [8]:
# HuggingFace datasets (safe & stable)
AI_DATASETS     = [{"name"      : "diffusiondb",
                    "hf_id"     : "poloclub/diffusiondb",
                    "config"    : "2m_first_1k",
                    "split"     : "train",
                    "image_key" : "image",
                    "label"     : "ai",
                    "family"    : "diffusion"
                  }]
        

REAL_DATASETS   = [{"name"      : "imagenette",
                    "hf_id"     : "frgfm/imagenette",
                    "config"    : "320px",
                    "split"     : "train",
                    "image_key" : "image",
                    "label"     : "real",
                    "family"    : "photographic",
                  }]

# Kaggle datasets (public, non-scraped)
KAGGLE_DATASETS = [{"name"      : "ai_vs_real",
                    "kaggle_id" : "tristanzhang32/ai-generated-images-vs-real-images",
                    "label"     : "ai",
                    "family"    : "mixed"
                   },
                   {"name"      : "midjourney",
                    "kaggle_id" : "cyanex1702/midjourney-imagesprompt",
                    "label"     : "ai",
                    "family"    : "diffusion"
                   }
                  ]


## HuggingFace Dataset Processor

Loads datasets via `datasets.load_dataset()` and saves images in unified format.

In [9]:
def process_hf_dataset(ds_cfg, root_dir, writer):
    print(f"\n▶ Loading HF dataset: {ds_cfg['name']}")

    ds      = load_dataset(ds_cfg["hf_id"],
                           **ds_cfg.get("hf_kwargs", {}),
                           name      = ds_cfg.get("config"),
                           split     = ds_cfg["split"],
                           streaming = ds_cfg.get("streaming", False),
                          )

    out_dir = root_dir / ds_cfg["name"]
    out_dir.mkdir(parents=True, exist_ok=True)

    count   = 0
    
    for row in tqdm(ds):
        if (count >= TARGET_PER_DS):
            break

        try:
            image = row.get(ds_cfg["image_key"])
            if not isinstance(image, Image.Image):
                continue

            uid   = uuid.uuid4().hex
            path  = out_dir / f"{uid}.png"

            normalize_and_save(image, path)

            write_meta(writer,
                       id       = uid,
                       filename = str(path),
                       label    = ds_cfg["label"],
                       family   = ds_cfg["family"],
                       source   = ds_cfg["name"],
                      )

            count += 1

        except Exception:
            continue
            

## Kaggle Dataset Downloader

Requires:
- Kaggle account
- ~/.kaggle/kaggle.json configured

No scraping. Fully legal.

In [10]:
def download_kaggle_dataset(kaggle_id: str, out_dir: Path):
    out_dir.mkdir(parents = True, exist_ok = True)

    if any(out_dir.iterdir()):
        print(f"✔ Kaggle dataset already present: {kaggle_id}")
        return

    print(f"⬇ Downloading Kaggle dataset: {kaggle_id}")

    subprocess.run(["kaggle", "datasets", "download",
                    kaggle_id,
                    "-p", str(out_dir),
                    "--unzip"
                   ],
                   check = True,
                  )


## Folder Ingestor

Converts **any folder of images** into the unified dataset format. 
Used for Kaggle & future web sources.

In [11]:
IMAGE_EXTS = {".png", ".jpg", ".jpeg", ".webp"}

def ingest_image_folder(src_dir, out_dir, writer, label, family, source):
    images = [p for p in src_dir.rglob("*") if p.suffix.lower() in IMAGE_EXTS]

    out_dir.mkdir(parents = True, exist_ok = True)

    for image_path in tqdm(images[:TARGET_PER_DS]):
        try:
            image = Image.open(image_path)

            uid   = uuid.uuid4().hex
            dst   = out_dir / f"{uid}.png"

            normalize_and_save(image, dst)

            write_meta(writer,
                       id       = uid,
                       filename = str(dst),
                       label    = label,
                       family   = family,
                       source   = source,
                      )
            
        except Exception:
            continue
            

## Main Pipeline Execution

This cell:
- Builds directories
- Processes HF datasets
- Downloads & ingests Kaggle datasets
- Writes unified metadata CSV

In [12]:
def main():
    ensure_dirs()

    with open(META_FILE, "w", newline = "") as f:
        writer = csv.DictWriter(f, fieldnames=["id", "filename", "label", "family", "source"])
        writer.writeheader()

        # HuggingFace datasets
        for ds in AI_DATASETS:
            process_hf_dataset(ds, AI_DIR, writer)

        for ds in REAL_DATASETS:
            process_hf_dataset(ds, REAL_DIR, writer)

        # Kaggle datasets
        for ds in KAGGLE_DATASETS:
            raw_path = RAW_DIR / ds["name"]
            download_kaggle_dataset(ds["kaggle_id"], raw_path)

            # AI images
            ingest_image_folder(src_dir = raw_path / "ai",
                                out_dir = AI_DIR / ds["name"],
                                writer  = writer,
                                label   = "ai",
                                family  = ds["family"],
                                source  = ds["name"],
                               )

            # REAL images
            ingest_image_folder(src_dir = raw_path / "real",
                                out_dir = REAL_DIR / ds["name"],
                                writer  = writer,
                                label   = "real",
                                family  = "photographic",
                                source  = ds["name"],
                               )

    print("\n✅ Dataset build complete")
    print(f"📄 Metadata saved at: {META_FILE}")


# ===============================
# Entry Point
# ===============================
if __name__ == "__main__":
    main()



▶ Loading HF dataset: diffusiondb


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [05:30<00:00,  3.02it/s]



▶ Loading HF dataset: imagenette


Using the latest cached version of the module from /Users/itobuz/.cache/huggingface/modules/datasets_modules/datasets/frgfm--imagenette/38929285b8abcae5c1305418e9d8fea5dd6b189bbbd22caba5f5537c7fa0f01f (last modified on Mon Dec 22 15:06:36 2025) since it couldn't be found locally at frgfm/imagenette., or remotely on the Hugging Face Hub.


Generating train split:   0%|          | 0/9469 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3925 [00:00<?, ? examples/s]

 11%|██████████████▋                                                                                                                            | 1000/9469 [02:30<21:13,  6.65it/s]


⬇ Downloading Kaggle dataset: tristanzhang32/ai-generated-images-vs-real-images


Traceback (most recent call last):
  File "/Users/itobuz/.conda/envs/mvp_env/bin/kaggle", line 7, in <module>
    sys.exit(main())
  File "/Users/itobuz/.conda/envs/mvp_env/lib/python3.10/site-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
  File "/Users/itobuz/.conda/envs/mvp_env/lib/python3.10/site-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
  File "/Users/itobuz/.conda/envs/mvp_env/lib/python3.10/site-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
KeyError: 'username'


CalledProcessError: Command '['kaggle', 'datasets', 'download', 'tristanzhang32/ai-generated-images-vs-real-images', '-p', '../tests/dataset/raw_downloads/ai_vs_real', '--unzip']' returned non-zero exit status 1.

# Post-Processing Attack Generator

This notebook applies **real-world post-processing attacks** to an existing
image dataset to evaluate robustness of AI-image detectors.

### Attacks Implemented
- JPEG recompression (quality loss)
- Resize / rescale (down + up)
- Gaussian blur

### Why this matters
Most AI images in the wild are:
- Screenshot
- Re-encoded
- Uploaded to social media
- Slightly blurred or resized

If a detector fails here, it fails in production.

## Imports & Config

In [ ]:
# ===============================
# Imports
# ===============================

import csv
import uuid
from PIL import Image
from tqdm import tqdm
from io import BytesIO
from pathlib import Path
from PIL import ImageFilter


# ===============================
# Configuration
# ===============================

BASE_DIR       = Path("tests/dataset")
ATTACK_DIR     = BASE_DIR / "attacked"
META_IN        = BASE_DIR / "metadata/dataset_index.csv"
META_OUT       = BASE_DIR / "metadata/dataset_index_attacked.csv"

ATTACK_DIR.mkdir(parents=True, exist_ok=True)

JPEG_QUALITIES = [95, 75, 50]
RESIZE_SCALES  = [0.75, 0.5]
BLUR_RADII     = [0.8, 1.5]


## Load Existing Metadata

We read the existing unified dataset index and create
new samples **derived from originals**.

In [ ]:
def load_metadata(path):
    with open(path, newline="") as f:
        return list(csv.DictReader(f))


records = load_metadata(META_IN)
print(f"Loaded {len(records)} original samples")


## Attack Primitives

Each function:
- Takes a PIL Image
- Returns a new PIL Image
- Does **not** modify the original

In [ ]:
def jpeg_attack(image: Image.Image, quality: int) -> Image.Image:
    """
    Simulate JPEG recompression
    """
    buf = BytesIO()
    image.save(buf, 
               format  = "JPEG", 
               quality = quality,
              )
    
    buf.seek(0)
    return Image.open(buf).convert("RGB")


def resize_attack(image: Image.Image, scale: float) -> Image.Image:
    """
    Downscale and upscale image
    """
    w, h         = image.size
    new_w, new_h = int(w * scale), int(h * scale)
    image_small  = image.resize((new_w, new_h), Image.BICUBIC)
    
    return image_small.resize((w, h), Image.BICUBIC)


def blur_attack(image: Image.Image, radius: float) -> Image.Image:
    """
    Apply Gaussian blur
    """
    return image.filter(ImageFilter.GaussianBlur(radius))
    

## Attack Application Pipeline

For each original image:
- Apply all attack variants
- Save attacked images
- Write **attack-aware metadata**

In [ ]:
def apply_attacks(records, writer):
    for r in tqdm(records):
        src_path = Path(r["filename"])
        
        if not src_path.exists():
            continue

        try:
            img = Image.open(src_path).convert("RGB")
            
        except Exception:
            continue

        base_name = src_path.stem
        label     = r["label"]

        out_base  = ATTACK_DIR / r["source"]
        out_base.mkdir(parents = True, exist_ok = True)

        # --- JPEG ---
        for q in JPEG_QUALITIES:
            attacked = jpeg_attack(img, q)
            uid      = uuid.uuid4().hex
            out_path = out_base / f"{uid}.png"

            attacked.save(out_path, optimize = True)

            writer.writerow({**r,
                             "id"        : uid,
                             "filename"  : str(out_path),
                             "attack"    : f"jpeg_q{q}",
                             "parent_id" : r["id"]
                           })

        # --- Resize ---
        for s in RESIZE_SCALES:
            attacked = resize_attack(img, s)
            uid      = uuid.uuid4().hex
            out_path = out_base / f"{uid}.png"

            attacked.save(out_path, optimize = True)

            writer.writerow({**r,
                             "id"        : uid,
                             "filename"  : str(out_path),
                             "attack"    : f"resize_{int(s*100)}",
                             "parent_id" : r["id"]
                           })

        # --- Blur ---
        for b in BLUR_RADII:
            attacked = blur_attack(img, b)
            uid      = uuid.uuid4().hex
            out_path = out_base / f"{uid}.png"

            attacked.save(out_path, optimize = True)

            writer.writerow({**r,
                             "id"        : uid,
                             "filename"  : str(out_path),
                             "attack"    : f"blur_{b}",
                             "parent_id" : r["id"]
                           })


## Write Attack Metadata

We preserve:
- Original label (ai / real)
- Source family
- Parent image ID
- Attack type

This allows **per-attack evaluation later**.

In [ ]:
with open(META_OUT, "w", newline = "") as f:
    fieldnames = list(records[0].keys()) + ["attack", "parent_id"]
    writer     = csv.DictWriter(f, fieldnames = fieldnames)
    writer.writeheader()

    apply_attacks(records, writer)

print("✅ Post-processing attacks generated")
print(f"Metadata saved to: {META_OUT}")
